In [1]:
import numpy as np
wordsList = np.load('wordsList.npy')
print('Loaded the word list!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
SeqLength = 42
counter = 0
for p in wordsList:
    counter = counter+1
    if counter >= SeqLength:                
        break

print("---------------------------------------")
wordVectors = np.load('wordVectors.npy')
print ('Loaded the word vectors!')

Loaded the word list!
---------------------------------------
Loaded the word vectors!


In [2]:
maxSeqLength = 10 #Maximum length of sentence
numDimensions = 300 #Dimensions for each word vector
inputSentence = np.zeros((maxSeqLength), dtype='int32')
inputSentence[0] = wordsList.index("happiness")
inputSentence[1] = wordsList.index("fear")
inputSentence[2] = wordsList.index("anger")
inputSentence[3] = wordsList.index("sadness")
inputSentence[4] = wordsList.index("disgust")
inputSentence[5] = wordsList.index("surprise")
inputSentence[6] = wordsList.index("neutral")
#inputSentence[7],inputSentence[8] and inputSentence[9] are going to be 0
print(inputSentence.shape)
print(inputSentence) #Shows the row index for each word

(10,)
[ 9811  1655  3946 13866 18741  2661  5876     0     0     0]


In [3]:
import tensorflow as tf
with tf.Session() as sess:
    print(tf.nn.embedding_lookup(wordVectors,inputSentence).eval())

[[ 0.039495    1.5316     -0.39991999 -0.20091     1.37839997  0.23264
   0.40810999  0.09424     0.80492997  1.18900001  0.37090001  0.29229999
   0.051458   -0.91470999  0.79913002  0.031613    0.23425999 -0.078071
   0.55264997 -0.094033   -0.067781    0.94492     0.13196    -0.2352
   1.61380005 -0.31953001 -1.30379999 -0.33394     1.0244      0.12196
   1.58580005  1.08249998 -0.15639    -0.38266999 -0.76346999 -0.49954
  -0.52539998 -0.33105999  0.29729    -0.17441    -0.076043   -0.83092999
  -0.47857001 -0.029247   -0.26328999 -0.30849999  0.15287    -0.30546999
   0.15899    -0.30671   ]
 [ 0.53661001 -0.33871999  0.28771999 -0.72583997 -0.041209    0.28498
   0.21664999  0.62408    -0.1204      0.41135001  0.10111    -0.11545
  -0.15651999 -0.39265001  0.66285998  0.71562999  0.42140001 -0.37041
  -0.081016   -0.29010001 -0.28406999  0.30706999  0.035974    0.32001999
   0.45456001 -2.08299994 -0.65890002 -0.081918    1.0431      0.50839001
   2.54329991  0.81831998 -0.11508 

In [4]:
import pandas as pd
file1 = pd.read_csv('text_emotion.csv')

In [5]:
labelledColumn = file1['label']

In [6]:
ll = labelledColumn.replace("worry","sadness")
ll = ll.replace("love","happiness")
ll = ll.replace("fun","happiness")
ll = ll.replace("enthusiasm","happiness")
ll = ll.replace("empty","sadness")
ll = ll.replace("boredom","neutral")
ll = ll.replace("relief","happiness")
ll = ll.replace("hate","anger")

In [7]:
textList1 = file1['text']

In [8]:
counterLabel = []
counterText = []
count=-1
for word in ll:
    count=count+1
    if word=="happiness" or word=="sadness" or word=="neutral":
        counterLabel.append(word)
        str1 = textList1[count]
        counterText.append(str1)

In [9]:
counterLabel[3]

'happiness'

In [10]:
counterText[3]

'wants to hang out with friends SOON!'

In [11]:
import re

def cleanSentences(string):
    string = string.lower().replace("\n", " ")
    string = string.replace("@", "")
    string = string.replace("=", "")
    string = string.replace("[", "")
    string = string.replace("#", "")
    string = string.replace("&quot;", "")
    return string.lower()

In [12]:
numWords1 = []
count1=0
for pf in counterText:
    counter1 = len(pf)
    numWords1.append(counter1)
    if(counter1 > 170):
        count1=count1+1
    
print('files read finished')

numFiles1 = len(numWords1)
print('The total number of files is', numFiles1)
print('The total number of words in the files is', sum(numWords1))
print('The average number of words in the files is', sum(numWords1)/len(numWords1))
print(count1)

files read finished
The total number of files is 35979
The total number of words in the files is 2627588
The average number of words in the files is 73.0311570638428
0


In [16]:
maxSeqLength = 170

In [17]:
ids = np.zeros((numFiles1, maxSeqLength), dtype='int32')
fileCounter = 0
for pf in counterText:
    indexCounter = 0
    for word in pf:
        try:
            ids[fileCounter][indexCounter] = wordsList.index(word)
        except ValueError:
            ids[fileCounter][indexCounter] = 399999 #Vector for unknown words
        indexCounter = indexCounter + 1
        if indexCounter >= maxSeqLength:
            break
    fileCounter = fileCounter + 1 
    
np.save('Data/idsMatrix123', ids)

In [18]:
ids = np.load('Data/idsMatrix123.npy')

In [19]:
ids.shape

(35979, 170)

In [42]:
from random import randint

def getTrainBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1,29999)
        labeltxt = counterLabel[num]
        labelindex = wordsList.index(labeltxt)
        arr[i] = ids[num-1:num]
        if labeltxt=="happiness":
            labels.append([1,0,0])
        elif labeltxt=="sadness":
            labels.append([0,1,0])
        elif labeltxt=="neutral":
            labels.append([0,0,1])
    return arr, labels

def getTestBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(30000,35978)
        labeltxt = counterLabel[num]
        print(labeltxt)
        labelemo = counterText[num]
        print(labelemo)
        labelindex = wordsList.index(labeltxt)
        arr[i] = ids[num-1:num]
        if labeltxt=="happiness":
            labels.append([1,0,0])
        elif labeltxt=="sadness":
            labels.append([0,1,0])
        elif labeltxt=="neutral":
            labels.append([0,0,1])
    return arr, labels

In [21]:
batchSize = 24
lstmUnits = 64
numClasses = 3
iterations = 100000

In [22]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

In [23]:
data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)
data.shape

TensorShape([Dimension(24), Dimension(170), Dimension(50)])

In [24]:
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

In [25]:
weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

In [26]:
correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [27]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [28]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [29]:
# sess = tf.InteractiveSession()
# saver = tf.train.Saver()
# sess.run(tf.global_variables_initializer())

# for i in range(iterations):
#    #Next Batch of reviews
#    nextBatch, nextBatchLabels = getTrainBatch();
#    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
   
#    #Write summary to Tensorboard
#    if (i % 50 == 0):
#        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
#        writer.add_summary(summary, i)

#    #Save the network every 10,000 training iterations
#    if (i % 10000 == 0 and i != 0):
#        save_path = saver.save(sess, "file1FilteredModels123/pretrained_lstm.ckpt", global_step=i)
#        print("saved to %s" % save_path)
# writer.close()

saved to file1FilteredModels123/pretrained_lstm.ckpt-10000
saved to file1FilteredModels123/pretrained_lstm.ckpt-20000
saved to file1FilteredModels123/pretrained_lstm.ckpt-30000
saved to file1FilteredModels123/pretrained_lstm.ckpt-40000
saved to file1FilteredModels123/pretrained_lstm.ckpt-50000
saved to file1FilteredModels123/pretrained_lstm.ckpt-60000
saved to file1FilteredModels123/pretrained_lstm.ckpt-70000
saved to file1FilteredModels123/pretrained_lstm.ckpt-80000
saved to file1FilteredModels123/pretrained_lstm.ckpt-90000


In [30]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('file1FilteredModels123'))

INFO:tensorflow:Restoring parameters from file1FilteredModels123\pretrained_lstm.ckpt-90000


In [44]:
iterations = 50
for i in range(iterations):
    nextBatch, nextBatchLabels = getTestBatch();
    print("Accuracy for this batch:", (sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100)
    print("---------------------------------------------------------------------------------------------------------------------")

neutral
@DoctorWatsonSx Morning
happiness
my miss doesn't read my tweets so she wouldn't know that the emo guy won #dsds
happiness
heeey heeey!   about to go read...
neutral
had a nice time with juno  http://plurk.com/p/sv71z
neutral
@Werecat1 good to know   thanks
happiness
I just received a mothers day card from my lovely daughter wishing my a happy mothers day  http://tr.im/kWK9
happiness
it's the last day  in lonesometown
sadness
Less than 99 days til warped tour.
neutral
@CynthiaY29 Nice info, I forwarded this to a needy friend of mine
neutral
@nikicheong try installing Twibble which is a Java based app that should work on most phones
sadness
@queetsss thanks babe. I guess I will find the right person one day, til then I'm better off alone
happiness
@TBHA aww , its ok,we ended up getting in later than expected and didnt go...I would of called you if we were able to, ...C U N Austin,
happiness
a night alone with my piano...and a pro tools session... ill take that ANYTIME.
sadness
@

Accuracy for this batch: 12.5
---------------------------------------------------------------------------------------------------------------------
happiness
@iMBA haha Good job getting close. Her eyes are set somewhere else though
happiness
Thanks @TheCampain for shoutin out my mom today  She also says thanks!
neutral
@mandirudge  just listened to the songs on your myspace.  really good
neutral
@giladby In rural #thailand, 2630 is considered posh
happiness
time for mimi's
happiness
@chantelleaustin  Good one
happiness
Our God is an awesome God, He is a God of 2nd,30th,100th chances..I give Him ALL the glory for my 2nd chance at 'love'.. Am still stunned
neutral
in the process of learning how to drive stick shift. its getting easier thanks to @mrjimson
happiness
@Mumbleguy We need men to give their input too so I'll fill u in on the time very soon
sadness
@joooo No, but Geek Minds Think Alike.
neutral
@Lynne_Lyons You get championship points for the heats too
happiness
Happy Mommy's  D

happiness
@09Casper oooooo. i like that too! anywhere near mommy is good for me.
happiness
@ukcatalogue fabulous combination till A. It arrives, B. The bill comes! Fun chatting to you. Have a good Sunday!
neutral
@theEmzi hï¿½rlich!
happiness
@steveplunkett  Hey, I need a car too. I'll dm you my address.
happiness
@andyclemmensen haha well then maybe u shouldn't be so weak  hehe jks  xx
happiness
watching finding nemo with my nephew so cute!
sadness
http://twitpic.com/4wkzi - before prom. this is when all the chaos began
sadness
Recogns he could get his assignments out of the way by June  bring it on
happiness
@hollymcombsbr somehow i cant reply to your message LOL and yes i know  Thank youuu
happiness
@error505 - well i greatly appreciate your comment.  That means a lot to me.  I'm working on both of them...work in progress, you know.
neutral
band practice
sadness
oh thats hard-going. I'm writing a book at the moment so am motivating myself to press on with that.
sadness
@faulko1 was 

Accuracy for this batch: 25.0
---------------------------------------------------------------------------------------------------------------------
neutral
@TheWineVault oh I know. and I aprreciate it  it can be my back up plan
happiness
@mismile you're so cute.
sadness
@faulko1 was always told to keep sucking something like fruit gums
neutral
using my twitter account
happiness
@Jason_Manford Awww thats such a lovely thing for u to say to me  are u doing any in manchester? x
happiness
Just taking my dad in law for a Sunday stroll down Old Leigh  a beautiful morning, say Hi to Morris LOL  http://twitpic.com/4wt8k
happiness
is resting. ahhhhh. I feel good
sadness
@DawnRichard OMG!! I done died and went 2 heaven...thanks so much 4 the response...I'm cryin' right now...Luv ya!!
neutral
@TheDigitalEric I'll believe it when I see it
happiness
The sun is shining brightly and the day has just begun! What's in store? Wandering around les calanques maybe? Vin, definitely!
happiness
Tiesto in Vic

Accuracy for this batch: 16.6666671634
---------------------------------------------------------------------------------------------------------------------
happiness
my legs are soft  watching a move with my mommm short day @ workk
happiness
http://twitpic.com/4wmoa - yeyy..
happiness
Got me a dark red hair dye, going to get some bleach
neutral
@justinbrighten Happy Birthday!
neutral
Woke up at 7:50 then fell back to sleep. Woke up at 8:50 and back to sleep again. Woke up at 9:50 and I'm staying awake! MORNING!
sadness
Off to see the Tinyï¿½s, thinking thoughts about smiles I caught from the radio  and big daft grin all over my face....
happiness
Happy Mothers day  &lt;3
neutral
@icebergstorm hey! negative on the primatech, this handle's been on the tubes for a decade longer than heroes has
neutral
@RoryCoaster btw, do you realize your profile pic makes you look much older than you say you are?
happiness
rsvp-ed qiuqiu's wedding invitation
neutral
@Noufah make it full girl 120%
happin

Accuracy for this batch: 0.0
---------------------------------------------------------------------------------------------------------------------
neutral
http://ping.fm/2UrGP (MPAA shows how to videorecord a TV)    via @Despil
happiness
@natzini 16lbs is still a huge achievement though
happiness
@ohaiijess it would be cool if they had people make videos of themselves miming the song and make one big video  so EVERYONE can be in it.
happiness
@yongfook  ack ~ of course i'm a person! but i think @alaskamiller knows that
happiness
@sumtxkid loved our lil tent-side jam session/mosh pit!!! EDGEFESSSSSST!!!
neutral
@mileycyruuus yes I can talk friench and in my picture this is me in france !!  Now , i'm in london and soon i'm going to L.A
sadness
@teksone No, I'm in the US for the weekend not the UK
happiness
@thebirchtree i'll try sort that fabric out for you today...been a busy bee!
happiness
watching snl with my ucsd girls and laughing about twitter/blackberry jokes  thought i'd tweet th

Accuracy for this batch: 4.16666679084
---------------------------------------------------------------------------------------------------------------------
happiness
..but did i mention spending time with two bffs .. Totally worth it
neutral
@Lesley_M Awww, our Mother's day has been and gone :o(.  No work today, off out witht he camera in a little while, hopefully
neutral
Thanks @ClaudiaMagic   ? http://blip.fm/~5z6n8
sadness
rain drops keep falling on my head..
happiness
@reags_ that's why twitter rocks!
neutral
@Tinkersdamn ah yours is better! enjoy
sadness
@AmyPredsfan Oops..too late.  Already popped in Bridges, since I missed most of it on tv.   (I &lt;3 Blair &amp; Jo too!)
happiness
@HockeyTShirt Folks thought it was hilarious when I told them the story, and then they saw the shirt I wore tonight.  Laughter all night.
happiness
learning to play guitar is one of the best things I have done out of boredom
happiness
@bluecat7  i definitely will , although with the caveat  that i no

Accuracy for this batch: 20.8333328366
---------------------------------------------------------------------------------------------------------------------
neutral
Prom is so over rated!! Irritated... Going to bed goodnight
neutral
@james_a_michael i plan on it! Goodnight
happiness
Happy Mothers day to all the Mom's on there!!
sadness
@ezrabutler - they have their headaches the morning after, instead!
sadness
is predicting a heavyweight battle between Lakers and Cleveland on the Finals! Well, hopefully! *crosses his fingers*
happiness
ever been in a pointless argument with drunk mum, drunk nan and drunk mums bf while having dinner??? i have.... fun times all round  hahah
happiness
@mrated Yeehah, quality
happiness
Happy Mothers Day to all the beautiful mom's out there  A day to cherish the best women in our lives
happiness
Yay Happy Mother's Day  #fb
happiness
@johncmayer ur really the sweetest person ever! thanks for making everyone's dreams come true..(p.s) my dream is for u 2twitte

Accuracy for this batch: 29.1666656733
---------------------------------------------------------------------------------------------------------------------
happiness
last sunday before going home!
neutral
Just woke up  shopping today for my birthday....kinda
happiness
Bout to go to bed... Happy Mother's Day ladies .... especially all th 1s who takin care of thier kids alone..n I Love U Momma
neutral
http://www.twitpic.com/4wp26  i made a cake for my mom  she loved it ? yay
happiness
I still can't believe the Matt McCoy interview got 255 views!! Twitter truly is....AWESOME!
sadness
@_ahseya_ cool i want you to take a look at this and tell me what you think
happiness
@halian: Happy Mother's day to my Mother. Thank you for being such an awesome mother
happiness
@rasga Yep: http://bit.ly/15yyId  We go there for Christmas treats!
neutral
@from10to300mm gut
happiness
Gossip girl &amp; pizza on the way = thank you to my lil babe
neutral
@vivianchiu LOL. You should buy your mummy flowers
happ

In [37]:
import matplotlib.pyplot as plt

In [45]:
x = ["a","b","c","d","e"]
y=[6,7,8,9,10]

In [ ]:
N = 5
men_means = (20, 35, 30, 35, 27)
men_std = (2, 3, 4, 1, 2)

ind = np.arange(N)  # the x locations for the groups
width = 0.35       # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(ind, men_means, width, color='r', yerr=men_std)

women_means = (25, 32, 34, 20, 25)
women_std = (3, 5, 2, 3, 3)
rects2 = ax.bar(ind + width, women_means, width, color='y', yerr=women_std)

# add some text for labels, title and axes ticks
ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(('G1', 'G2', 'G3', 'G4', 'G5'))

ax.legend((rects1[0], rects2[0]), ('Men', 'Women'))


def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height),
                ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

plt.show()

In [1]:
import pydicom 